# Классификация поведения клиентов телеком оператора

In [1]:
import pandas as pd
import numpy as np 

from catboost import Pool
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.dummy import DummyClassifier

## [1. Описание данных](#data_review)

In [2]:
users_df = pd.read_csv('./datasets/users_behavior.csv')

In [3]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
users_df.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Данные без пропусков и предобработка не требуется.

## [2. Разделение выборки](#train_split)

Так как спрятанная тестовая выборка отсутствует, то исходные датасет следует разбить в пропорциях 60/20/20.

In [5]:
df_train, df_valid = train_test_split(users_df, test_size=0.4, shuffle=True, random_state=123456)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, shuffle=True, random_state=123456)

In [6]:
def preprocess_data_part(data, column_drop):
    features = data.drop([column_drop], axis=1)
    target = data[column_drop]
    
    return features, target
    
features_train, target_train = preprocess_data_part(data=df_train, column_drop='is_ultra')
features_valid, target_valid = preprocess_data_part(data=df_valid, column_drop='is_ultra')
features_test, target_test = preprocess_data_part(data=df_test, column_drop='is_ultra')

Целевым признаком в данном датасете является столбец данных `is_ultra`. 

## [3. Исследование качества разных моделей.](#models_research)
### [3.1 Классификатор дерева решений.](#decision_tree_classifier)

In [7]:
def train_decision_tree_classifier(features, target, max_depth, random_state, **kwargs):
    model = DecisionTreeClassifier(random_state=random_state, max_depth=max_depth, **kwargs)
    model.fit(features, target)
    
    return model 

In [8]:
def get_score_decision_tree_classifier(features_train, target_train, features_valid, target_valid, max_param, random_state):
    train_score = {}
    valid_score = {}

    for param in range(1, max_param):
        for split_ in range(2, max_param):
            for leaf_ in range(1, max_param):
                model = train_decision_tree_classifier(features=features_train, target=target_train, max_depth=param, random_state=random_state, min_samples_split=split_, min_samples_leaf=leaf_)

                train_score[f'{param}_{split_}_{leaf_}'] = model.score(features_train, target_train)
                valid_score[f'{param}_{split_}_{leaf_}'] = model.score(features_valid, target_valid)

    train_score = {param: score for param, score in sorted(train_score.items(), key=lambda item: item[1], reverse=True)}
    valid_score = {param: score for param, score in sorted(valid_score.items(), key=lambda item: item[1], reverse=True)}

    top_train_param, top_train_score = list(train_score.items())[0]
    top_valid_param, top_valid_score = list(valid_score.items())[0]
    
    return {'params_train': top_train_param, 'score_train': top_train_score, 'params_valid': top_valid_param, 'score_valid': top_valid_score} 

In [9]:
decision_tree_score = get_score_decision_tree_classifier(features_train, target_train, features_valid, target_valid, max_param=21, random_state=123456)

In [10]:
print(f'Лучший результат на обучающей выборке: с параметрами = '
      f'{decision_tree_score["params_train"]}, результат = {decision_tree_score["score_train"]}')

print(f'Лучший результат на валидационной выборке: с параметрами = '
      f'{decision_tree_score["params_valid"]}, результат = {decision_tree_score["score_valid"]}')

Лучший результат на обучающей выборке: с параметрами = 20_2_1, результат = 0.9880705394190872
Лучший результат на валидационной выборке: с параметрами = 5_2_12, результат = 0.8118195956454122


Исследовал классификатор *дерева решений*, лучший результат `0.8118195956454122` на валидационной выборке получил с параметрами **максимальной глубины дерева** `(max_depth)` **= 5**, **минимальным количеством примеров для разделения** `(min_samples_split)` **= 2** и **минимальным количеством объектов в листе** `(min_samples_leaf)` **= 12** точность.

### [3.2 Классификатор случайного леса](#random_forest_classifier)

In [11]:
def train_random_forest_classifier(features, target, n_estimators, random_state, **kwargs):
    model = RandomForestClassifier(n_estimators=n_estimators, random_state=123456, **kwargs)
    model.fit(features, target)
    
    return model 

In [12]:
def get_score_random_forest_classifier(features_train, 
                                       target_train, 
                                       features_valid, 
                                       target_valid, 
                                       start_estimators, max_estimators, step_estimators, 
                                       start_depth, max_depth, step_depth,
                                       random_state, **kwargs):
    train_score = {}
    valid_score = {}
    
    for estimator in range(start_estimators, max_estimators, step_estimators):
        for depth in range(start_depth, max_depth, step_depth):
            model = train_random_forest_classifier(features=features_train, target=target_train, 
                                                   n_estimators=estimator, max_depth=depth, 
                                                   random_state=random_state, **kwargs)
        
            train_score[f'{estimator}_{depth}'] = model.score(features_train, target_train)
            valid_score[f'{estimator}_{depth}'] = model.score(features_valid, target_valid)
      
    train_score = {param: score for param, score in sorted(train_score.items(), key=lambda item: item[1], reverse=True)}
    valid_score = {param: score for param, score in sorted(valid_score.items(), key=lambda item: item[1], reverse=True)}

    top_train_param, top_train_score = list(train_score.items())[0]
    top_valid_param, top_valid_score = list(valid_score.items())[0]
    
    return {'params_train': top_train_param, 'score_train': top_train_score, 'params_valid': top_valid_param, 'score_valid': top_valid_score} 

In [13]:
random_forest_score = get_score_random_forest_classifier(features_train, target_train, 
                                                         features_valid, target_valid, 
                                                         start_estimators=1, max_estimators=257, step_estimators=10,
                                                         start_depth=1, max_depth=21, step_depth=1,
                                                         random_state=123456) 

In [14]:
print(f'Лучший результат на обучающей выборке: с параметрами = '
      f'{random_forest_score["params_train"]}, результат = {random_forest_score["score_train"]}')

print(f'Лучший результат на валидационной выборке: с параметрами = '
      f'{random_forest_score["params_valid"]}, результат = {random_forest_score["score_valid"]}')

Лучший результат на обучающей выборке: с параметрами = 41_20, результат = 0.9979253112033195
Лучший результат на валидационной выборке: с параметрами = 91_8, результат = 0.8195956454121306


Исследовал классификатор *случайного леса*, лучший результат `0.8195956454121306` на валидационной выборке получил с **количеством деревьев** `(n_estimators)` **= 91**, **максимальной глубиной дерева** `(max_depth)` **= 8**.

### [3.3 Классификатор логической регрессии.](#logistic_regression)

In [15]:
logistic_regression_model = LogisticRegression(random_state=123456, solver='lbfgs')
logistic_regression_model.fit(features_train, target_train)

logistic_regression_train_score = logistic_regression_model.score(features_train, target_train)
logistic_regression_valid_score = logistic_regression_model.score(features_valid, target_valid)

In [16]:
print(f'Результат на обучающей выборке: {logistic_regression_train_score}')
print(f'Результат на валидационной выборке: {logistic_regression_valid_score}')

Результат на обучающей выборке: 0.7411825726141079
Результат на валидационной выборке: 0.7402799377916018


Результат применения классификации методом *логической регрессии* равен `0.7402799377916018` на валидационной выборке.

### [3.4 Другие методы классификации.](#other_classification)

In [18]:
neigh_train_score = {}
neigh_valid_score = {}

for eighbor in range (1, 21):
    neigh_model = KNeighborsClassifier(n_neighbors=eighbor)
    neigh_model.fit(features_train, target_train)

    neigh_train_score[eighbor] = neigh_model.score(features_train, target_train)
    neigh_valid_score[eighbor] = neigh_model.score(features_valid, target_valid)

In [19]:
neigh_train = {estimators: score for estimators, score in sorted(neigh_train_score.items(), key=lambda item: item[1], reverse=True)}
neigh_valid = {estimators: score for estimators, score in sorted(neigh_valid_score.items(), key=lambda item: item[1], reverse=True)}

neigh_train_param, neigh_train_score = list(neigh_train.items())[0]
neigh_valid_param, neigh_valid_score = list(neigh_valid.items())[0]

print(f'Результат на обучающей выборке: с параметрами = {neigh_train_param}, результат = {neigh_train_score}')
print(f'Результат на валидационной выборке: с параметрами = {neigh_valid_param}, результат = {neigh_valid_score}')

Результат на обучающей выборке: с параметрами = 1, результат = 1.0
Результат на валидационной выборке: с параметрами = 6, результат = 0.7511664074650077


Результат применения метода *K-ближайших соседей* **с количеством соседей** `n_neighbors` **= 6** равен `0.7511664074650077` на валидационной выборке.

In [20]:
gauss_train_score = {}
gauss_valid_score = {}

gauss_model = GaussianNB()
gauss_model.fit(features_train, target_train)

gauss_train_score = gauss_model.score(features_train, target_train)
gauss_valid_score = gauss_model.score(features_valid, target_valid)

print(f'Результат на обучающей выборке: {gauss_train_score}')
print(f'Результат на валидационной выборке: {gauss_valid_score}')

Результат на обучающей выборке: 0.7847510373443983
Результат на валидационной выборке: 0.7729393468118196


Результат применения метода *Gaussian Naive Bayes (GaussianNB)* равен `0.7729393468118196` на валидационной выборке.

In [21]:
train_pool = Pool(features_train, target_train)
valid_pool = Pool(features_valid, target_valid)

cat_model = CatBoostClassifier(iterations=1000, learning_rate=0.05, eval_metric='Accuracy', random_state=123456)
cat_model.fit(train_pool, eval_set=valid_pool, verbose=100)

0:	learn: 0.7795643	test: 0.7651633	best: 0.7651633 (0)	total: 52.2ms	remaining: 52.1s
100:	learn: 0.8329876	test: 0.8211509	best: 0.8258165 (92)	total: 249ms	remaining: 2.22s
200:	learn: 0.8651452	test: 0.8071540	best: 0.8258165 (92)	total: 445ms	remaining: 1.77s
300:	learn: 0.8947095	test: 0.8102644	best: 0.8258165 (92)	total: 644ms	remaining: 1.5s
400:	learn: 0.9133817	test: 0.7978227	best: 0.8258165 (92)	total: 831ms	remaining: 1.24s
500:	learn: 0.9263485	test: 0.7947123	best: 0.8258165 (92)	total: 1.04s	remaining: 1.03s
600:	learn: 0.9377593	test: 0.7884914	best: 0.8258165 (92)	total: 1.34s	remaining: 893ms
700:	learn: 0.9481328	test: 0.7838258	best: 0.8258165 (92)	total: 1.59s	remaining: 679ms
800:	learn: 0.9548755	test: 0.7838258	best: 0.8258165 (92)	total: 1.89s	remaining: 469ms
900:	learn: 0.9616183	test: 0.7807154	best: 0.8258165 (92)	total: 2.18s	remaining: 240ms
999:	learn: 0.9725104	test: 0.7744946	best: 0.8258165 (92)	total: 2.37s	remaining: 0us

bestTest = 0.8258164852
b

In [22]:
cat_valid_score = cat_model.score(features_valid, target_valid)

Результат применения классификатора *CatBoostClassifier* равен `0.8258164852` на валидационной выборке.

## [4. Проверка моделей на тестовой выборке.](#testing_models)
### [4.1 Тестирование модели дерева решений.](#test_decision_tree_classifier)

In [23]:
decision_tree_params = [int(param) for param in decision_tree_score['params_valid'].split('_')]

print(f'Параметры модели дерева решений: max_depth={decision_tree_params[0]}, ' 
      f'min_samples_split={decision_tree_params[1]}, ' 
      f'min_samples_leaf={decision_tree_params[2]}')

Параметры модели дерева решений: max_depth=5, min_samples_split=2, min_samples_leaf=12


In [24]:
decision_tree_model = train_decision_tree_classifier(features=features_train, target=target_train, 
                                                     max_depth=decision_tree_params[0], 
                                                     min_samples_split=decision_tree_params[1], 
                                                     min_samples_leaf=decision_tree_params[2],
                                                     random_state=123456)

decision_tree_test_score = decision_tree_model.score(features_test, target_test)

In [25]:
print(f'Результат модели дерева решений на валидационной выборке: {decision_tree_score["score_valid"]}')
print(f'Результат модели дерева решений на тестовой выборке: {decision_tree_test_score}')
print(f'Расхождение модели {(decision_tree_score["score_valid"] - decision_tree_test_score)}')

Результат модели дерева решений на валидационной выборке: 0.8118195956454122
Результат модели дерева решений на тестовой выборке: 0.8118195956454122
Расхождение модели 0.0


Проверка модели дерева решений на тестовой выборке показала аналогичные результаты.

### [4.2 Тестирование модели случайного леса.](#test_random_forest_classifier)

In [26]:
random_forest_params = [int(param) for param in random_forest_score['params_valid'].split('_')]

print(f'Параметры модели случайного дерева: n_estimators={random_forest_params[0]}, ' 
      f'max_depth={random_forest_params[1]}')

Параметры модели случайного дерева: n_estimators=91, max_depth=8


In [27]:
random_forest_model = train_random_forest_classifier(features=features_train, target=target_train, 
                                                     n_estimators=random_forest_params[0], max_depth=random_forest_params[1],
                                                     random_state=123456)

random_forest_test_score = random_forest_model.score(features_test, target_test)

In [28]:
print(f'Результат модели дерева решений на валидационной выборке: {random_forest_score["score_valid"]}')
print(f'Результат модели дерева решений на тестовой выборке: {random_forest_test_score}')
print(f'Расхождение модели {(random_forest_score["score_valid"] - random_forest_test_score):0.2%}')

Результат модели дерева решений на валидационной выборке: 0.8195956454121306
Результат модели дерева решений на тестовой выборке: 0.8180404354587869
Расхождение модели 0.16%


Проверка модели случайного леса на тестовой выборке показала небольшое расхождение с результатами на валидационной выборке.

### [4.3 Тестирование модели логической регрессии.](#test_logistic_regression)

In [29]:
logistic_regression_test_score = logistic_regression_model.score(features_test, target_test)

In [30]:
print(f'Результат модели логической регрессии на валидационной выборке: {logistic_regression_valid_score}')
print(f'Результат модели логической регрессии на тестовой выборке: {logistic_regression_test_score}')
print(f'Расхождение модели {(logistic_regression_valid_score - logistic_regression_test_score):0.2%}')

Результат модели логической регрессии на валидационной выборке: 0.7402799377916018
Результат модели логической регрессии на тестовой выборке: 0.7651632970451011
Расхождение модели -2.49%


Проверка модели логической регрессии на тестовой выборке показала небольшое расхождение с результатами на валидационной выборке.

### [4.4 Тестирование других методов классификации.](#test_other_classification)

In [34]:
print(f'Параметры модели K-ближайших соседей: n_neighbors={eighbor}')

Параметры модели K-ближайших соседей: n_neighbors=20


In [35]:
kneighbors_model = KNeighborsClassifier(n_neighbors=eighbor)
kneighbors_model.fit(features_train, target_train)

kneighbors_test_score = kneighbors_model.score(features_test, target_test)

In [36]:
print(f'Результат модели K-ближайших соседей на валидационной выборке: {neigh_valid_score}')
print(f'Результат модели K-ближайших соседей на тестовой выборке: {kneighbors_test_score}')
print(f'Расхождение модели {(neigh_valid_score - kneighbors_test_score):0.2%}')

Результат модели K-ближайших соседей на валидационной выборке: 0.7511664074650077
Результат модели K-ближайших соседей на тестовой выборке: 0.7822706065318819
Расхождение модели -3.11%


Проверка модели K-ближайших соседей на тестовой выборке показала небольшое расхождение с результатами на валидационной выборке.

In [37]:
gauss_test_score = gauss_model.score(features_test, target_test)

print(f'Результат модели Gaussian Naive Bayes на валидационной выборке: {gauss_valid_score}')
print(f'Результат модели Gaussian Naive Bayes на тестовой выборке: {gauss_test_score}')
print(f'Расхождение модели {(gauss_valid_score - gauss_test_score):0.2%}')

Результат модели Gaussian Naive Bayes на валидационной выборке: 0.7729393468118196
Результат модели Gaussian Naive Bayes на тестовой выборке: 0.7900466562986003
Расхождение модели -1.71%


Проверка модели Gaussian Naive Bayes на тестовой выборке показала небольшое расхождение с результатами на валидационной выборке.

In [38]:
cat_test_score = cat_model.score(features_test, target_test)

print(f'Результат модели CatBoostClassifier на валидационной выборке: {cat_valid_score}')
print(f'Результат модели CatBoostClassifier на тестовой выборке: {cat_test_score}')
print(f'Расхождение модели {(cat_valid_score - cat_test_score):0.2%}')

Результат модели CatBoostClassifier на валидационной выборке: 0.8258164852255054
Результат модели CatBoostClassifier на тестовой выборке: 0.8149300155520995
Расхождение модели 1.09%


Проверка модели Gaussian Naive Bayes на тестовой выборке показала небольшое расхождение с результатами на валидационной выборке.

In [39]:
models_rank = {'CatBoostClassifier': cat_test_score, 
               'GaussianNB': gauss_test_score, 
               'KNeighborsClassifier': kneighbors_test_score, 
               'LogisticRegression': logistic_regression_test_score, 
               'RandomForestClassifier': random_forest_test_score, 
               'DecisionTreeClassifier': decision_tree_test_score}

models_rank = {model_name: score for model_name, score in sorted(models_rank.items(), key=lambda item: item[1], reverse=True)}

In [40]:
models_rank

{'RandomForestClassifier': 0.8180404354587869,
 'CatBoostClassifier': 0.8149300155520995,
 'DecisionTreeClassifier': 0.8118195956454122,
 'GaussianNB': 0.7900466562986003,
 'KNeighborsClassifier': 0.7822706065318819,
 'LogisticRegression': 0.7651632970451011}

In [41]:
print(f'Разница между двумя наибольшими результатами: {(list(models_rank.values())[0] - list(models_rank.values())[1]):0.2%}')

Разница между двумя наибольшими результатами: 0.31%


### [5. Проверка моделей на вменяемость.](#sanity_check_models)

Для проверки моделей на можно сравнить со случайной моделью.

In [42]:
dummy_clf = DummyClassifier(strategy='uniform')
dummy_clf.fit(features_train, target_train)

dummy_valid = dummy_clf.score(features_valid, target_valid)
dummy_test = dummy_clf.score(features_test, target_test)

print(f'Результат случайной модели на тестовой выборке: {dummy_test}')

Результат случайной модели на тестовой выборке: 0.4976671850699845


In [43]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(features_train, target_train)

dummy_valid = dummy_clf.score(features_valid, target_valid)
dummy_test = dummy_clf.score(features_test, target_test)

print(f'Результат случайной модели на тестовой выборке: {dummy_test}')

Результат случайной модели на тестовой выборке: 0.7216174183514774


Результат случайной модели на тестовой выборке равен примерно 50%, что доказывает вменяемость всех рассмотренных моделей. По результатам тестированиям модели можно выделить две с наибольшими результатами. Это случайны лес `RandomForestClassifier` и CatBoost `CatBoostClassifier`. Так как разница между ними не велика и скорость обучения модели CatBoost выше, то данная модель наиболее предпочтительна.